In [1]:
import numpy as np
import pandas as pd 
import numpy.random as rn
from tqdm import tqdm_notebook as tqdm

from scipy import optimize       # to compare

/home/alice/Documents/ramax_vacation/venv/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
import pickle 

with open('possible_vacation.pickle', 'rb') as f:
    possible_vacation = pickle.load(f)

with open('req_work.pickle', 'rb') as f:
    req_work = pickle.load(f)
    
with open('excel_dict.pickle', 'rb') as f:
    excel_dict = pickle.load(f)

with open('qs.pickle', 'rb') as f:
    qs = pickle.load(f)

In [3]:
def annealing(random_start,
              cost_function,
              random_neighbour,
              acceptance,
              temperature,
              args,
              maxsteps=1000,
              debug=True
             ):
    state = random_start(*args)
    cost = cost_function(state)
    states, costs = [state], [cost]
    for step in range(maxsteps):
        fraction = step / float(maxsteps)
        T = temperature(fraction)
        new_state = random_neighbour(*args)  # fraction
        new_cost = cost_function(new_state)
#         if debug: print("Step #{:>2}/{:>2} : T = {:>4.3g}, state = {:>4.3g}, cost = {:>4.3g}, new_state = {:>4.3g}, new_cost = {:>4.3g} ...".format(step, maxsteps, T, state, cost, new_state, new_cost))
        if debug: print("Step #{:>2}/{:>2} : T = {:>4.3g} cost = {:>4.3g} new_cost = {:>4.3g} ...".format(step, maxsteps, T, cost, new_cost))          
        if acceptance_probability(cost, new_cost, T) > rn.random():
            state, cost = new_state, new_cost
            states.append(state)
            costs.append(cost)
    return state, cost_function(state), states, costs

In [41]:
def get_random_vac(max_fly_person, max_fly_month):
    return np.random.randint(36, min(max_fly_person, max_fly_month))


def random_matrix(df, q_name):
    state = np.zeros((df.shape[0], 12), dtype=int)
    max_fly_by_month = get_max_fly_by_month()
    aval_hours_by_month = get_aval_hours_by_month(df)

    for month in range(12):
        i = 0
        while aval_hours_by_month.loc[q_name, month] > 0:
            i += 1
            if state.shape[0] - i < 0:
                break
            rand_i = np.random.randint(0, state.shape[0])

            _vac = get_random_vac(
                max_fly_person=df['Max Fly'].iloc[rand_i],
                max_fly_month=max_fly_by_month[month]
            )

            if state[rand_i][month] > 0:
                continue

            # Если уже использовали больше чем нужно отпуск
            if state[rand_i].sum() + _vac > max(df['Max Fly'].iloc[rand_i] * 2, 184):
                continue

            # Если у нас на расстоянии 2 уже есть отпуск
            if month < 11:
                if state[rand_i][month + 1] > 0 or state[rand_i][month - 1] > 0:
                    continue
            else:
                if state[rand_i][0] > 0 or state[rand_i][10] > 0:
                    continue

            # Если число доступных часов больше
            if aval_hours_by_month.loc[q_name, month] >= _vac:
                aval_hours_by_month.loc[q_name, month] -= _vac
                state[rand_i, month] = _vac

            # Если число доступных часов меньше, то переносим отпуск на след месяц
            else:
                delta = _vac - aval_hours_by_month.loc[q_name, month]
                state[rand_i, month] = aval_hours_by_month.loc[q_name, month]
                aval_hours_by_month.loc[q_name, month] = 0

                if month < 11:
                    state[rand_i, month + 1] += delta
                    aval_hours_by_month.loc[q_name, month + 1] -= delta
                else:
                    state[rand_i, 0] += delta
                    aval_hours_by_month.loc[q_name, 0] -= delta
                break
    return state

In [42]:
possible_vacation

,0,1,2,3,4,5,6,7,8,9,10,11
Q1,8744.0,11250.0,6669.0,6941.0,11465.0,3163.0,3643.0,3940.0,4790.0,6266.0,9006.0,5103.0
Q2,6221.0,7151.0,6040.0,6178.0,6205.0,7021.0,6223.0,6253.0,6654.0,6519.0,7681.0,7196.0
Q3,2591.0,3521.0,2410.0,2548.0,2575.0,3391.0,2593.0,2623.0,3024.0,2889.0,4051.0,3566.0
Q4,767.0,3526.0,4067.0,3194.0,3361.0,1275.0,1813.0,1351.0,845.0,5295.0,2609.0,1897.0
Q5,873.0,3053.0,3386.0,3261.0,2637.0,1221.0,1742.0,1273.0,938.0,3288.0,2808.0,1980.0
Q6,9961.0,8308.0,9699.0,9504.0,11801.0,3802.0,5262.0,4606.0,4715.0,10796.0,6201.0,4953.0
Q7,1846.0,1686.0,1564.0,965.0,1323.0,840.0,777.0,1152.0,1309.0,694.0,1047.0,953.0
Q8,1036.0,1431.0,1331.0,998.0,1195.0,794.0,951.0,846.0,1000.0,1150.0,978.0,916.0
Q9,751.0,2913.0,1270.0,1768.0,2009.0,852.0,722.0,719.0,831.0,1766.0,1108.0,1114.0
Q10,4098.0,4753.0,4205.0,4702.0,3872.0,3601.0,3317.0,3317.0,3592.0,3361.0,3719.0,3446.0


In [43]:
def get_max_fly_by_month():
    return excel_dict['months'][2].values

In [44]:
def get_D_d(vacs):
    return vacs.sum(axis=0)

def get_D(df, q_name, by_month=False):
    if not by_month:
        return possible_vacation.loc[q_name, :].sum()
    else:
        return possible_vacation.loc[q_name, :].values.copy()

def get_aval_hours_by_month(df):
    res = possible_vacation.copy()
    res.columns = range(12)
    return res
    
def get_S(df):
    return df[[f'RP{i}_score' for i in range(1, 13)]].values.sum().sum()
    
def get_S_d(vacs, df):
    a = df[[f'RP{i}_score' for i in range(1, 13)]].values
    b = vacs.astype(bool).astype(int)
    return np.multiply(a, b).sum().sum()
    
def cost_function(vacs):
    return f(vacs)

In [45]:
def cost_function(vacs):
    cost = 1 - get_S_d(vacs, df) / get_S(df)
    return cost

In [46]:
def acceptance_probability(cost, new_cost, temperature):
    if new_cost < cost:
        return 1
    else:
        p = np.exp(- (new_cost - cost) / temperature)
        return p

In [47]:
def temperature(fraction):
    return max(0.01, min(1, 1 - fraction))

In [48]:
def create_output(q_name, state, df):

    l = []

    for vac, person, person_score in zip(state, df['Num'].values, df[[f'RP{i}_score' for i in range(1, 13)]].values):
        for i, (vac_size, score) in enumerate(zip(vac, person_score), 1):
            if vac_size != 0:
                l.append([person, i, vac_size, 1 if score else 0])
    
    _df = pd.DataFrame(data=l, columns=['id', 'month', 'rest_size', 'aplication'])
    _df.to_csv(f'../data/res_v.0.03_{q_name}.csv')

In [49]:
for q_name, df in qs.items():
    print(q_name)
    state, c, states, costs = annealing(
        random_matrix,
        cost_function,
        random_matrix,
        acceptance_probability,
        temperature,
        args=[df, q_name],
        maxsteps=100,
        debug=True
    )
    create_output(q_name, state, df)

Q1
Step # 0/100 : T =    1 cost = 0.714 new_cost = 0.708 ...
Step # 1/100 : T = 0.99 cost = 0.708 new_cost = 0.723 ...
Step # 2/100 : T = 0.98 cost = 0.723 new_cost = 0.729 ...
Step # 3/100 : T = 0.97 cost = 0.729 new_cost = 0.729 ...
Step # 4/100 : T = 0.96 cost = 0.729 new_cost = 0.736 ...
Step # 5/100 : T = 0.95 cost = 0.736 new_cost = 0.711 ...
Step # 6/100 : T = 0.94 cost = 0.711 new_cost = 0.733 ...
Step # 7/100 : T = 0.93 cost = 0.733 new_cost = 0.707 ...
Step # 8/100 : T = 0.92 cost = 0.707 new_cost = 0.715 ...
Step # 9/100 : T = 0.91 cost = 0.715 new_cost = 0.709 ...
Step #10/100 : T =  0.9 cost = 0.709 new_cost = 0.733 ...
Step #11/100 : T = 0.89 cost = 0.733 new_cost = 0.739 ...
Step #12/100 : T = 0.88 cost = 0.739 new_cost = 0.704 ...
Step #13/100 : T = 0.87 cost = 0.704 new_cost = 0.721 ...
Step #14/100 : T = 0.86 cost = 0.721 new_cost = 0.732 ...
Step #15/100 : T = 0.85 cost = 0.732 new_cost = 0.729 ...
Step #16/100 : T = 0.84 cost = 0.729 new_cost = 0.739 ...
Step #17/10

Step #44/100 : T = 0.56 cost = 0.725 new_cost = 0.757 ...
Step #45/100 : T = 0.55 cost = 0.757 new_cost = 0.734 ...
Step #46/100 : T = 0.54 cost = 0.734 new_cost = 0.726 ...
Step #47/100 : T = 0.53 cost = 0.726 new_cost = 0.71 ...
Step #48/100 : T = 0.52 cost = 0.71 new_cost = 0.729 ...
Step #49/100 : T = 0.51 cost = 0.729 new_cost = 0.713 ...
Step #50/100 : T =  0.5 cost = 0.713 new_cost = 0.751 ...
Step #51/100 : T = 0.49 cost = 0.751 new_cost = 0.809 ...
Step #52/100 : T = 0.48 cost = 0.751 new_cost = 0.747 ...
Step #53/100 : T = 0.47 cost = 0.747 new_cost = 0.713 ...
Step #54/100 : T = 0.46 cost = 0.713 new_cost = 0.675 ...
Step #55/100 : T = 0.45 cost = 0.675 new_cost = 0.768 ...
Step #56/100 : T = 0.44 cost = 0.768 new_cost = 0.718 ...
Step #57/100 : T = 0.43 cost = 0.718 new_cost = 0.699 ...
Step #58/100 : T = 0.42 cost = 0.699 new_cost = 0.675 ...
Step #59/100 : T = 0.41 cost = 0.675 new_cost = 0.703 ...
Step #60/100 : T =  0.4 cost = 0.703 new_cost = 0.725 ...
Step #61/100 : T

Step #86/100 : T = 0.14 cost = 0.707 new_cost = 0.708 ...
Step #87/100 : T = 0.13 cost = 0.708 new_cost = 0.691 ...
Step #88/100 : T = 0.12 cost = 0.691 new_cost = 0.626 ...
Step #89/100 : T = 0.11 cost = 0.626 new_cost = 0.714 ...
Step #90/100 : T =  0.1 cost = 0.714 new_cost = 0.694 ...
Step #91/100 : T = 0.09 cost = 0.694 new_cost = 0.711 ...
Step #92/100 : T = 0.08 cost = 0.711 new_cost = 0.733 ...
Step #93/100 : T = 0.07 cost = 0.733 new_cost = 0.688 ...
Step #94/100 : T = 0.06 cost = 0.688 new_cost = 0.766 ...
Step #95/100 : T = 0.05 cost = 0.688 new_cost = 0.688 ...
Step #96/100 : T = 0.04 cost = 0.688 new_cost = 0.717 ...
Step #97/100 : T = 0.03 cost = 0.688 new_cost = 0.665 ...
Step #98/100 : T = 0.02 cost = 0.665 new_cost = 0.723 ...
Step #99/100 : T = 0.01 cost = 0.665 new_cost = 0.715 ...
Q4
Step # 0/100 : T =    1 cost = 0.716 new_cost = 0.704 ...
Step # 1/100 : T = 0.99 cost = 0.704 new_cost = 0.713 ...
Step # 2/100 : T = 0.98 cost = 0.713 new_cost = 0.711 ...
Step # 3/10

Step #28/100 : T = 0.72 cost = 0.739 new_cost = 0.708 ...
Step #29/100 : T = 0.71 cost = 0.708 new_cost = 0.698 ...
Step #30/100 : T =  0.7 cost = 0.698 new_cost = 0.69 ...
Step #31/100 : T = 0.69 cost = 0.69 new_cost = 0.759 ...
Step #32/100 : T = 0.68 cost = 0.759 new_cost = 0.672 ...
Step #33/100 : T = 0.67 cost = 0.672 new_cost = 0.764 ...
Step #34/100 : T = 0.66 cost = 0.764 new_cost = 0.733 ...
Step #35/100 : T = 0.65 cost = 0.733 new_cost = 0.664 ...
Step #36/100 : T = 0.64 cost = 0.664 new_cost = 0.692 ...
Step #37/100 : T = 0.63 cost = 0.692 new_cost = 0.741 ...
Step #38/100 : T = 0.62 cost = 0.741 new_cost = 0.723 ...
Step #39/100 : T = 0.61 cost = 0.723 new_cost = 0.718 ...
Step #40/100 : T =  0.6 cost = 0.718 new_cost = 0.721 ...
Step #41/100 : T = 0.59 cost = 0.721 new_cost = 0.703 ...
Step #42/100 : T = 0.58 cost = 0.703 new_cost = 0.723 ...
Step #43/100 : T = 0.57 cost = 0.723 new_cost = 0.662 ...
Step #44/100 : T = 0.56 cost = 0.662 new_cost = 0.754 ...
Step #45/100 : T

Step #70/100 : T =  0.3 cost = 0.707 new_cost = 0.731 ...
Step #71/100 : T = 0.29 cost = 0.731 new_cost = 0.736 ...
Step #72/100 : T = 0.28 cost = 0.736 new_cost = 0.732 ...
Step #73/100 : T = 0.27 cost = 0.732 new_cost = 0.711 ...
Step #74/100 : T = 0.26 cost = 0.711 new_cost = 0.74 ...
Step #75/100 : T = 0.25 cost = 0.74 new_cost = 0.711 ...
Step #76/100 : T = 0.24 cost = 0.711 new_cost = 0.737 ...
Step #77/100 : T = 0.23 cost = 0.737 new_cost = 0.739 ...
Step #78/100 : T = 0.22 cost = 0.739 new_cost = 0.704 ...
Step #79/100 : T = 0.21 cost = 0.704 new_cost = 0.718 ...
Step #80/100 : T =  0.2 cost = 0.718 new_cost = 0.712 ...
Step #81/100 : T = 0.19 cost = 0.712 new_cost = 0.738 ...
Step #82/100 : T = 0.18 cost = 0.738 new_cost = 0.747 ...
Step #83/100 : T = 0.17 cost = 0.747 new_cost = 0.73 ...
Step #84/100 : T = 0.16 cost = 0.73 new_cost = 0.724 ...
Step #85/100 : T = 0.15 cost = 0.724 new_cost = 0.735 ...
Step #86/100 : T = 0.14 cost = 0.735 new_cost = 0.735 ...
Step #87/100 : T =

Step #12/100 : T = 0.88 cost = 0.686 new_cost = 0.765 ...
Step #13/100 : T = 0.87 cost = 0.765 new_cost = 0.756 ...
Step #14/100 : T = 0.86 cost = 0.756 new_cost = 0.758 ...
Step #15/100 : T = 0.85 cost = 0.758 new_cost = 0.735 ...
Step #16/100 : T = 0.84 cost = 0.735 new_cost = 0.758 ...
Step #17/100 : T = 0.83 cost = 0.758 new_cost = 0.73 ...
Step #18/100 : T = 0.82 cost = 0.73 new_cost = 0.733 ...
Step #19/100 : T = 0.81 cost = 0.733 new_cost = 0.785 ...
Step #20/100 : T =  0.8 cost = 0.785 new_cost = 0.762 ...
Step #21/100 : T = 0.79 cost = 0.762 new_cost = 0.726 ...
Step #22/100 : T = 0.78 cost = 0.726 new_cost = 0.739 ...
Step #23/100 : T = 0.77 cost = 0.739 new_cost = 0.726 ...
Step #24/100 : T = 0.76 cost = 0.726 new_cost = 0.756 ...
Step #25/100 : T = 0.75 cost = 0.726 new_cost = 0.725 ...
Step #26/100 : T = 0.74 cost = 0.725 new_cost = 0.76 ...
Step #27/100 : T = 0.73 cost = 0.76 new_cost = 0.754 ...
Step #28/100 : T = 0.72 cost = 0.754 new_cost = 0.808 ...
Step #29/100 : T =

Step #54/100 : T = 0.46 cost = 0.765 new_cost = 0.74 ...
Step #55/100 : T = 0.45 cost = 0.74 new_cost = 0.626 ...
Step #56/100 : T = 0.44 cost = 0.626 new_cost = 0.772 ...
Step #57/100 : T = 0.43 cost = 0.772 new_cost = 0.779 ...
Step #58/100 : T = 0.42 cost = 0.779 new_cost = 0.717 ...
Step #59/100 : T = 0.41 cost = 0.717 new_cost = 0.733 ...
Step #60/100 : T =  0.4 cost = 0.733 new_cost = 0.674 ...
Step #61/100 : T = 0.39 cost = 0.674 new_cost = 0.743 ...
Step #62/100 : T = 0.38 cost = 0.743 new_cost = 0.626 ...
Step #63/100 : T = 0.37 cost = 0.626 new_cost = 0.745 ...
Step #64/100 : T = 0.36 cost = 0.745 new_cost = 0.669 ...
Step #65/100 : T = 0.35 cost = 0.669 new_cost = 0.726 ...
Step #66/100 : T = 0.34 cost = 0.726 new_cost = 0.725 ...
Step #67/100 : T = 0.33 cost = 0.725 new_cost = 0.628 ...
Step #68/100 : T = 0.32 cost = 0.628 new_cost = 0.745 ...
Step #69/100 : T = 0.31 cost = 0.628 new_cost = 0.722 ...
Step #70/100 : T =  0.3 cost = 0.722 new_cost = 0.71 ...
Step #71/100 : T 

Step #96/100 : T = 0.04 cost = 0.688 new_cost = 0.777 ...
Step #97/100 : T = 0.03 cost = 0.688 new_cost = 0.706 ...
Step #98/100 : T = 0.02 cost = 0.706 new_cost = 0.777 ...
Step #99/100 : T = 0.01 cost = 0.706 new_cost = 0.678 ...
